<a href="https://colab.research.google.com/github/Otobi1/Back-to-Basics-A-Refresher-/blob/master/Back_to_Basics_CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## Set up 

import numpy as np 
import pandas as pd
import random 
import torch 
import torch.nn as nn

In [4]:
SEED = 1234

In [5]:
def set_seeds(seed = 1234):
  """Set seed for reproducibility."""
  np.random.seed(seed)
  random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed) # multi GPU

In [6]:
# Set seed for reproducibility 

set_seeds(seed=SEED)

In [7]:
# Set device 

cuda = True
device = torch.device("cuda" if (
    torch.cuda.is_available() and cuda) else "cpu")
torch.set_default_tensor_type("torch.FloatTensor")
if device.type == "cuda":
  torch.set_default_tensor_type("torch.cuda.FloatTensor")
print (device)

cuda


In [8]:
# Load data 
# - corpus of news article from http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html 

url = "https://raw.githubusercontent.com/GokuMohandas/madewithml/main/datasets/news.csv"
df = pd.read_csv(url, header = 0) # load
df = df.sample(frac = 1).reset_index(drop = True) # shuffle
df.head()

,title,category
0,Sharon Accepts Plan to Reduce Gaza Army Operat...,World
1,Internet Key Battleground in Wildlife Crime Fight,Sci/Tech
2,July Durable Good Orders Rise 1.7 Percent,Business
3,Growing Signs of a Slowing on Wall Street,Business
4,The New Faces of Reality TV,World


In [9]:
# Preprocessing 
# - to clean up the data, convert to lower text, remove filler words and filter using regex.

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

In [10]:
nltk.download("stopwords")
STOPWORDS = stopwords.words("english")
print (STOPWORDS[:5])
porter = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we']


In [11]:
def preprocess(text, stopwords = STOPWORDS):
  """Conditional preprocessing on our text unique to the task."""
  # Lower 
  text = text.lower()

  # Remove stopwords
  pattern = re.compile(r"\b(" + r"|".join(stopwords) + r")\b\s*")
  text = pattern.sub(" ", text)

  # Remove words in parenthesis
  text = re.sub(r"\([^)]*\)", " ", text)

  # Spacing and filters 
  text = re.sub(r"([-;;.,!?<=>])", r" \1 ", text)
  text = re.sub('[^A-Za-z0-9]+', ' ', text) # remove non alphanumeric characters
  text = re.sub(' +', ' ', text) # remove multiple spaces 
  text = text.strip()

  return text

In [12]:
# Sample 

text = "Great week for the NYSE"
preprocess(text = text)

'great week nyse'

In [13]:
# Apply to dataframe

preprocessed_df = df.copy()
preprocessed_df.title = preprocessed_df.title.apply(preprocess)
print (f"{df.title.values[0]}\n\n{preprocessed_df.title.values[0]}")

Sharon Accepts Plan to Reduce Gaza Army Operation, Haaretz Says

sharon accepts plan reduce gaza army operation haaretz says


In [14]:
# if you have preprocessing steps like standardisation, that are calculated, you need to separate the training an dtest set first before applying those operations. 
# this is because we cannot apply any knowledge gained from the test set accidentally (data leak during preprocessing/training). 
# - for global preprocessing steps like the functin above, where we arent learning anything from the data itself, we can perform them before splitting the data.

In [15]:
# Split the data 

import collections
from sklearn.model_selection import train_test_split

In [16]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.15
TEST_SIZE = 0.15

In [17]:
def train_val_test_split(X, y, train_size):
  """Split dataset into data split."""
  X_train, X_, y_train, y_ = train_test_split(X, y, train_size = TRAIN_SIZE, stratify = y)
  X_val, X_test, y_val, y_test = train_test_split(X_, y_, train_size = 0.5, stratify = y_)
  return X_train, X_val, X_test, y_train, y_val, y_test

In [18]:
# Data
X = preprocessed_df["title"].values 
y = preprocessed_df["category"].values

In [19]:
# Create data splits
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X = X, y = y, train_size = TRAIN_SIZE)

print (f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print (f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print (f"X_train: {X_test.shape}, y_test: {y_test.shape}")
print (f"Sample point: {X_train[0]} -> {y_train[0]}")


X_train: (84000,), y_train: (84000,)
X_val: (18000,), y_val: (18000,)
X_train: (18000,), y_test: (18000,)
Sample point: china battles north korea nuclear talks -> World


In [20]:
# Label Encoding 
# to encode the text labels into unique indices

import itertools

In [21]:
class LabelEncoder(object):
  """Label encoder for tag labels."""
  def __init__(self, class_to_index = {}):
    self.class_to_index = class_to_index
    self.index_to_class = {v: k for k, v in self.class_to_index.items()}
    self.classes = list(self.class_to_index.keys())

  def __len__(self):
    return len(self.class_to_index)

  def __str__(self):
    return f"<LabelEncoder(num_classes = {len(self)})>"
  
  def fit(self, y):
    classes = np.unique(y_train)
    for i, class_ in enumerate(classes):
      self.class_to_index[class_] = i
    self.index_to_class = {v: k for k, v in self.class_to_index.items()}
    self.classes = list(self.class_to_index.keys())
    return self

  def encode(self, y):
    encoded = np.zeros((len(y)), dtype = int)
    for i, item in enumerate(y):
      encoded[i] = self.class_to_index[item]
    return encoded

  def decode(self, y):
    classes = []
    for i, item in enumerate(y):
      classes.append(self.index_to_class[item])
    return classes

  def save(self, fp):
    with open(fp, "w") as fp:
      contents = {"class_to_index": self.class_to_index}
      json.dump(contents, fp, indent = 4, sort_keys = False)

  @classmethod
  def load(cls, fp):
    with open(fp, "r") as fp:
      kwargs = json.load(fp = fp)
    return clas(**kwargs)

In [22]:
# Encode 

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
NUM_CLASSES = len(label_encoder)
label_encoder.class_to_index

{'Business': 0, 'Sci/Tech': 1, 'Sports': 2, 'World': 3}

In [23]:
# Converting labels to tokens 

print (f"y_train[0]: {y_train[0]}")

y_train = label_encoder.encode(y_train)
y_val = label_encoder.encode(y_val)
y_test = label_encoder.encode(y_test)

print (f"y_train[0]: {y_train[0]}")

y_train[0]: World
y_train[0]: 3


In [24]:
# Class weights 

counts = np.bincount(y_train)
class_weights = {i: 1.0/count for i, count in enumerate(counts)}
print (f"counts: {counts}\nweights: {class_weights}")

counts: [21000 21000 21000 21000]
weights: {0: 4.761904761904762e-05, 1: 4.761904761904762e-05, 2: 4.761904761904762e-05, 3: 4.761904761904762e-05}


In [25]:
# Tokenizer 

import json
from collections import Counter
from more_itertools import take

In [26]:
class Tokeniser(object):
  def __init__(self, char_level, num_tokens = None,
               pad_token = "<PAD>", oov_token = "<UNK>", 
               token_to_index = None):
    self.char_level = char_level
    self.separator = " " if self.char_level else " "
    if num_tokens: num_tokens -= 2 # pad + unk tokens
    self.num_tokens = num_tokens
    self.oov_token = oov_token
    if not token_to_index:
      token_to_index = {"<PAD>": 0, "<UNK>": 1}
    self.token_to_index = token_to_index
    self.index_to_token = {v: k for k, v in self.token_to_index.items()}

  def __len__(self):
    return len(self.token_to_index)

  def __str__(self):
    return f"<Tokeniser(num_tokens = {len(self)})>"

  def fit_on_texts(self, texts):
    if self.char_level:
      all_tokens = [token for text in texts for token in text]
    if not self.char_level:
      all_tokens = [token for text in texts for token in text.split(" ")]
    counts = Counter(all_tokens).most_common(self.num_tokens)
    self.min_token_freq = counts[-1][1]
    for token, count in counts:
      index = len(self)
      self.token_to_index[token] = index
      self.index_to_token[index] = token
    return self

  def texts_to_sequence(self, texts):
    sequences = []
    for text in texts:
      if not self.char_level:
        text = text.split(' ')
      sequence = []
      for token in text: 
        sequence.append(self.token_to_index.get(
            token, self.token_to_index[self.oov_token]))
      sequences.append(np.asarray(sequence))
    return sequences
  
  def sequences_to_texts(self, sequences):
    texts = []
    for sequence in sequences:
      text = []
      for index in sequence :
        text.append(self.index_to_token.get(index, self.oov_token))
      texts.append(self.separator.join([token for token in text]))
    return texts
  
  def save(self, fp):
    with open(fp, "w") as fp:
      contents = {
          "char_level": self.char_level, 
          "oov_token": self.oov_token, 
          "token_to_index": self.token_to_index
      }
      json.dump(contents, fp, indent = 4, sort_keys = False)

  @classmethod
  def load(cls, fp):
    with open(fp, "r") as fp:
      kwargs = json.load(fp = fp)
    return cls(**kwargs)

In [27]:
## - we will restrict the number of tokens in our tokenizer to the top 500 most frequent tokens (stop words already removed)
# -- because the full vocabulary (approx 30k) is too large to run on google colab

# ** it is important that we are only using the training data split because during inference, the model will not always know every token
# -- so it is important to replicate that scenario with the validation and test split. 

In [28]:
# Tokenise

tokeniser = Tokeniser(char_level = False, num_tokens = 500)
tokeniser.fit_on_texts(texts = X_train)
VOCAB_SIZE = len(tokeniser)

print (tokeniser)

<Tokeniser(num_tokens = 500)>


In [29]:
# Sample of tokens 

print (take(5, tokeniser.token_to_index.items()))
print (f"least freq tokens freq: {tokeniser.min_token_freq}") # use this to adjust num tokens

[('<PAD>', 0), ('<UNK>', 1), ('39', 2), ('b', 3), ('gt', 4)]
least freq tokens freq: 166


In [30]:
# Convert texts to sequences of indices 

X_train = tokeniser.texts_to_sequence(X_train)
X_val = tokeniser.texts_to_sequence(X_val)
X_test = tokeniser.texts_to_sequence(X_test)

preprocessed_text = tokeniser.sequences_to_texts([X_train[0]])[0]
print ("Text to indices: \n"
    f" (preprocessed) -> {preprocessed_text}\n"
    f" (tokenised) -> {X_train[0]}")

Text to indices: 
 (preprocessed) -> china <UNK> north korea nuclear talks
 (tokenised) -> [ 16   1 285 142 114  24]


In [31]:
# One-hot Encoding 

# - Creates a binary column fro each unique value of each feature. 
# -- All the values for the token will be 0 except the index of that specific token

In [32]:
def to_categorical(seq, num_classes):
  """One-hot encode a sequence of tokens."""
  one_hot = np.zeros((len(seq), num_classes))
  for i, item in enumerate(seq):
    one_hot[i, item] = 1.
  return one_hot

In [33]:
# One-hot encoding 
print (X_train[0])
print (len(X_train[0]))
cat = to_categorical(seq = X_train[0], num_classes = len(tokeniser))

print (cat)
print (cat.shape)

[ 16   1 285 142 114  24]
6
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6, 500)


In [34]:
# Convert tokens to one-hot

vocab_size = len(tokeniser)
X_train = [to_categorical(seq, num_classes = vocab_size) for seq in X_train]
X_val = [to_categorical(seq, num_classes = vocab_size) for seq in X_val]
X_test = [to_categorical(seq, num_classes = vocab_size) for seq in X_test]

In [35]:
# Padding 

# - all the inputs have varying lengths, but each batch needs to e uniformly shaped
# - we can use padding to make all the inputs in the batch the same length
# - the padding index will be 0

## ** one-hot encoding creates a batch of shape (N, max_seq_len, vocab_size) so we'll need to pad 3D sequences 

In [36]:
def pad_sequences(sequences, max_seq_len = 0):
  """Pad sequences to max length in sequence."""
  max_seq_len = max(max_seq_len, max(len(sequence) for sequence in sequences))
  num_classes = sequences[0].shape[-1]
  padded_sequences = np.zeros((len(sequences), max_seq_len, num_classes))
  for i, sequence in enumerate(sequences):
    padded_sequences[i][:len(sequence)] = sequence
  return padded_sequences

In [37]:
# 3D sequences 

print (X_train[0].shape, X_train[1].shape, X_train[2].shape)
padded = pad_sequences(X_train[0:3])
print (padded.shape)

(6, 500) (5, 500) (6, 500)
(3, 6, 500)


In [38]:
# Dataset
# - here we need to create datasets and dataloaders to be able to efficiently create batches with the data splits 

FILTER_SIZE = 1 # unigram

In [39]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, X, y, max_filter_size):
    self.X = X
    self.y = y
    self.max_filter_size = max_filter_size

  def __len__(self):
    return len(self.y)

  def __str__(self):
    return f"<Dataset(N = {len(self)})>"

  def __getitem__(self, index):
    X = self.X[index]
    y = self.y[index]
    return [X, y]

  def collate_fn(self, batch):
    """Processing on batch."""
    # Get inputs
    X = np.array(batch, dtype = object)[:, 0]
    y = np.stack(np.array(batch, dtype = object)[:, 1], axis = 0)

    # Pad sequences 
    X = pad_sequences(X, max_seq_len = self.max_filter_size)

    # Cast
    X = torch.FloatTensor(X.astype(np.int32))
    y = torch.LongTensor(y.astype(np.int32))

    return X, y

  def create_dataloader(self, batch_size, shuffle = False, drop_last = False):
    return torch.utils.data.DataLoader(
        dataset = self, batch_size = batch_size, collate_fn = self.collate_fn, 
        shuffle = shuffle, drop_last = drop_last, pin_memory = True)

In [40]:
# Create datasets for embedding 

train_dataset = Dataset(X = X_train, y = y_train, max_filter_size = FILTER_SIZE)
val_dataset = Dataset(X = X_val, y = y_val, max_filter_size = FILTER_SIZE)
test_dataset = Dataset(X = X_test, y = y_test, max_filter_size = FILTER_SIZE)
print ("Datasets: \n"
    f" Train dataset: {train_dataset.__str__()}\n"
    f" Val dataset: {val_dataset.__str__()}\n"
    f" Test dataset: {test_dataset.__str__()}\n"
    "Sample Point: \n"
    f" X: {test_dataset[0][0]}\n"
    f" y: {test_dataset[0][1]}")

Datasets: 
 Train dataset: <Dataset(N = 84000)>
 Val dataset: <Dataset(N = 18000)>
 Test dataset: <Dataset(N = 18000)>
Sample Point: 
 X: [[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
 y: 1


In [41]:
# Create dataloaders 

batch_size = 64
train_dataloader = train_dataset.create_dataloader(batch_size = batch_size)
val_dataloader = val_dataset.create_dataloader(batch_size = batch_size)
test_dataloader = test_dataset.create_dataloader(batch_size = batch_size)
batch_X, batch_y = next(iter(test_dataloader))
print ("Sample batch:\n"
    f" X: {list(batch_X.size())}\n"
    f" y: {list(batch_y.size())}\n"
    "Sample point:\n"
    f" X: {batch_X[0]}\n"
    f" y: {batch_y[0]}")

Sample batch:
 X: [64, 14, 500]
 y: [64]
Sample point:
 X: tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cpu')
 y: 1
